In [1]:
from prody import *
from pylab import *
import json

In [2]:
pdb, header = parsePDB('6acg', header=True)

@> PDB file is found in working directory (6acg.pdb).
@> 29715 atoms and 1 coordinate set(s) were parsed in 0.45s.


In [3]:
# get contacts
spike_contacts = pdb.select('chain A B C and within 5 of chain D')
ace2_contacts = pdb.select('chain D and within 5 of chain A B C')

# writePDB('spike_contacts.pdb', spike_contacts)
# writePDB('ace2_contacts.pdb', ace2_contacts)

In [4]:
spike = pdb.select('chain A B C')
ace2  = pdb.select('chain D')

In [5]:
interactions = findNeighbors(atoms=spike, radius=5, atoms2=ace2)

In [6]:
spike_atoms = []
ace2_atoms = []
links = []

In [7]:
# function to format atom data
def getAtomData(atom):
    data = {}

    data["chain"]   = atom.getData('chain')
    data["resname"] = atom.getData('resname')
    data["resnum"]  = int(atom.getData('resnum'))
    data["atom"] = atom.getData('name')
    data["name"]   = " ".join([data["chain"], data['resname'], str(data['resnum']), data['atom']])
    data["id"]   = atom.getIndex()

    return data

In [8]:
# store nodes and links 
for pair in interactions:
    a, b, distance = pair

    spike_atom = getAtomData(a)    
    ace2_atom  = getAtomData(b)

    spike_atoms.append(spike_atom)
    ace2_atoms.append(ace2_atom)

    links.append({
        "source": spike_atom["id"],
        "target": ace2_atom["id"]
    })

In [9]:
# remove duplicates in atom lists
spike_atoms_uniq = list({v['id']:v for v in spike_atoms}.values())
ace2_atoms_uniq  = list({v['id']:v for v in ace2_atoms}.values())

In [10]:
interaction_json = {
    "nodes": spike_atoms_uniq + ace2_atoms_uniq,
    "links": links
}

In [11]:
# write to json
with open('interactions.json', 'w') as file:
    json.dump(interaction_json, file)

In [12]:
# Just for verification
spike_contact_residues = sorted(spike_contacts.getData('resnum'))
spike_contact_residues2 = sorted([atom['resnum'] for atom in spike_atoms_uniq])
spike_contact_residues == spike_contact_residues2

True